In [ ]:
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Casadi


In [ ]:
# Dubins Car Model: x, y, theta
def dubins_dynamics(x, u, dt):
    x_next = ca.vertcat(
        x[0] + u[0] * ca.cos(x[2]) * dt,  # x_next = x + v * cos(theta) * dt
        x[1] + u[0] * ca.sin(x[2]) * dt,  # y_next = y + v * sin(theta) * dt
        x[2] + u[1] * dt,  # theta_next = theta + omega * dt
    )
    return x_next


# Define Optimization Problem
N = 20  # Horizon length
dt = 0.1  # Time step

# State & Control Variables
x = ca.MX.sym("x", 3)  # (x, y, theta)
u = ca.MX.sym("u", 2)  # (v, omega)

# Obstacles (x, y, radius)
obstacles = [(2.0, 2.0, 0.5), (4.0, 3.0, 0.7)]


def cost_function(x_seq, u_seq, goal):
    cost = 0
    for t in range(N):
        cost += ca.sumsqr(x_seq[:, t] - goal)  # Distance to goal
        cost += 0.1 * ca.sumsqr(u_seq[:, t])  # Smooth control effort
    return cost


# Optimization Variables
X = ca.MX.sym("X", 3, N + 1)
U = ca.MX.sym("U", 2, N)

# Constraints List
constraints = []
g = []

for t in range(N):
    x_next = dubins_dynamics(X[:, t], U[:, t], dt)
    constraints.append(X[:, t + 1] - x_next)  # Enforce dynamics
    for obs in obstacles:
        obs_x, obs_y, obs_r = obs
        constraints.append(
            (X[0, t] - obs_x) ** 2 + (X[1, t] - obs_y) ** 2 - obs_r**2
        )  # Avoid obstacle

# Flatten Constraints
g = ca.vertcat(*constraints)

# Solve the NLP
goal = np.array([5, 5, 0])  # Target waypoint
nlp = {"x": ca.vertcat(ca.vec(X), ca.vec(U)), "f": cost_function(X, U, goal), "g": g}
solver = ca.nlpsol("solver", "ipopt", nlp)

# Initial State & Bounds
x_init = np.array([0, 0, 0])
X0 = np.tile(x_init.reshape(-1, 1), (1, N + 1))
U0 = np.zeros((2, N))
lb_x = -ca.inf * np.ones((3, N + 1))  # No bounds on state
ub_x = ca.inf * np.ones((3, N + 1))
lb_u = np.array([[0.0, -1.0]]).T * np.ones((2, N))  # Min speed, max turn rate
ub_u = np.array([[1.0, 1.0]]).T * np.ones((2, N))

sol = solver(
    x0=ca.vertcat(ca.vec(X0), ca.vec(U0)),
    lbg=0,
    ubg=0,  # Constraints must be satisfied
    lbx=ca.vertcat(ca.vec(lb_x), ca.vec(lb_u)),
    ubx=ca.vertcat(ca.vec(ub_x), ca.vec(ub_u)),
)

# Extract Solution
x_opt = np.array(sol["x"][: 3 * (N + 1)]).reshape(3, N + 1)
u_opt = np.array(sol["x"][3 * (N + 1) :]).reshape(2, N)

print("Optimized Trajectory:", x_opt)
print("Optimized Controls:", u_opt)

# Plot Results
plt.figure(figsize=(8, 6))
plt.plot(x_opt[0, :], x_opt[1, :], "b-o", label="Optimized Path")
plt.scatter(x_init[0], x_init[1], c="g", marker="o", label="Start")
plt.scatter(goal[0], goal[1], c="r", marker="x", label="Goal")

# Plot obstacles
for obs in obstacles:
    circle = plt.Circle((obs[0], obs[1]), obs[2], color="r", alpha=0.5)
    plt.gca().add_patch(circle)

plt.xlabel("X position")
plt.ylabel("Y position")
plt.legend()
plt.grid()
plt.axis("equal")
plt.title("Dubins Trajectory Optimization")
plt.show()

# do-mpc


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import do_mpc
from casadi import SX, vertcat

In [ ]:
# Define model
def dubins_model():
    model_type = "continuous"
    model = do_mpc.model.Model(model_type)

    # States
    x = model.set_variable(var_type="_x", var_name="x", shape=(1, 1))
    y = model.set_variable(var_type="_x", var_name="y", shape=(1, 1))
    theta = model.set_variable(var_type="_x", var_name="theta", shape=(1, 1))

    # Controls
    v = model.set_variable(var_type="_u", var_name="v", shape=(1, 1))
    omega = model.set_variable(var_type="_u", var_name="omega", shape=(1, 1))

    # Dynamics
    model.set_rhs("x", v * np.cos(theta))
    model.set_rhs("y", v * np.sin(theta))
    model.set_rhs("theta", omega)

    model.setup()
    return model


# Initialize model
model = dubins_model()

# Define MPC controller
mpc = do_mpc.controller.MPC(model)
setup_mpc = {
    "n_horizon": 20,
    "t_step": 0.1,
    "state_discretization": "collocation",
    "store_full_solution": True,
}
mpc.set_param(**setup_mpc)

# Define cost function
cost_expr = (model.x["x"] - 5) ** 2 + (model.x["y"] - 5) ** 2
mpc.set_objective(
    mterm=cost_expr, lterm=cost_expr + 0.1 * (model.u["v"] ** 2 + model.u["omega"] ** 2)
)
mpc.set_rterm(v=0.1, omega=0.1)

# Constraints
mpc.bounds["lower", "_u", "v"] = 0.0
mpc.bounds["upper", "_u", "v"] = 1.0
mpc.bounds["lower", "_u", "omega"] = -1.0
mpc.bounds["upper", "_u", "omega"] = 1.0

# Obstacles
obstacles = [(2.0, 2.0, 0.5), (4.0, 3.0, 0.7)]
for obs_x, obs_y, obs_r in obstacles:
    mpc.set_nl_cons(
        "obs_avoid_{}_{}".format(obs_x, obs_y),
        (model.x["x"] - obs_x) ** 2 + (model.x["y"] - obs_y) ** 2 - (obs_r**2 + 0.1),
        ub=0.0,
    )

mpc.setup()

# Initial State
x0 = np.array([0, 0, 0])
mpc.x0 = x0
mpc.set_initial_guess()

# Simulation
simulator = do_mpc.simulator.Simulator(model)
simulator.set_param(t_step=0.1)
simulator.setup()
simulator.x0 = x0

n_steps = 50
x_traj = [x0]
for _ in range(n_steps):
    u = mpc.make_step(x_traj[-1])
    x_next = simulator.make_step(u).flatten()
    x_traj.append(x_next)

x_traj = np.array(x_traj)

# Plot Results
plt.figure(figsize=(8, 6))
plt.plot(x_traj[:, 0], x_traj[:, 1], "b-o", label="Optimized Path")
plt.scatter(x0[0], x0[1], c="g", marker="o", label="Start")
plt.scatter(5, 5, c="r", marker="x", label="Goal")

# Plot obstacles
for obs in obstacles:
    circle = plt.Circle((obs[0], obs[1]), obs[2], color="r", alpha=0.5)
    plt.gca().add_patch(circle)

plt.xlabel("X position")
plt.ylabel("Y position")
plt.legend()
plt.grid()
plt.axis("equal")
plt.title("Dubins Trajectory Optimization with do-mpc")
plt.show()